In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

from umucv.util import cameraOutline

from ipywidgets import interactive

def homog(x):
    ax = np.array(x)
    uc = np.ones(ax.shape[:-1]+(1,))
    return np.append(ax,uc,axis=-1)

# convierte en coordenadas tradicionales
def inhomog(x):
    ax = np.array(x)
    return ax[..., :-1] / ax[...,[-1]]

# juntar columnas
def jc(*args):
    return np.hstack(args)

# aplica una transformación homogénea h a un conjunto
# de puntos ordinarios, almacenados como filas 
def htrans(h,x):
    return inhomog(homog(x) @ h.T)

def rmsreproj(view, model, transf):
    err = view - htrans(transf,model)
    return np.sqrt(np.mean(err.flatten()**2))

def pose(K, image, model):
    ok,rvec,tvec = cv.solvePnP(model, image, K, (0,0,0,0))
    if not ok:
        return 1e6, None
    R,_ = cv.Rodrigues(rvec)
    M = K @ jc(R,tvec)
    rms = rmsreproj(image,model,M)
    return rms, M

# Matriz de calibración (ejemplo, completa con los valores reales de tu cámara)
K = np.array([[1667, 0, 969], 
              [0, 1666, 544], 
              [0, 0, 1]])

# Coordenadas de las esquinas del tablero en la imagen (en píxeles)
image_points = np.array([
    [1395, 286],   # Esquina superior izquierda
    [1636, 231],   # Esquina superior derecha
    [1637, 415],   # Esquina inferior derecha
    [1394, 442]    # Esquina inferior izquierda
], dtype='float32')

canasta = np.array([
    [0, 0, 3.95],   # Esquina superior izquierda
    [1.8, 0, 3.95],   # Esquina superior derecha
    [1.8, 0, 2.90],   # Esquina inferior derecha
    [0, 0, 2.90]    # Esquina inferior izquierda
], dtype='float32')

esquina = np.array([
    [12.5, 0.4, 0],
    [-9.5, 0.4, 0],
    [-9.5, -20.6,0]
], dtype='float32')


# Nuevo punto a añadir
nuevo_punto = np.array([[0, 0, 3.95]], dtype='float32')

# Agregar el nuevo punto
canasta_view = np.vstack([canasta, nuevo_punto])

# probamos todas las asociaciones de puntos imagen con modelo
# y nos quedamos con la que produzca menos error
def bestPose(K,view,model):
    poses = [ pose(K, v.astype(float), model) for v in rots(view) ]
    return sorted(poses,key=lambda p: p[0])[0]

def rots(c):
    return [np.roll(c,k,0) for k in range(len(c))]

err,Me = bestPose(K,image_points,canasta)
print(err)

pose(K, image_points, canasta)

camline = cameraOutline(Me)

# Definir el aro (radio estándar de un aro de baloncesto es 0.225 metros)
def crear_aro(num_puntos=16):
    radio = 0.225  # Radio estándar del aro de baloncesto (en metros)
    # El aro se ubica a 3.05 metros de altura y a cierta distancia del tablero
    centro_x = 0.9  # Centro del aro (mitad del ancho del tablero)
    centro_y = - 0.376  # Distancia que sobresale del tablero
    centro_z = 3.05  # Altura reglamentaria
    
    puntos_aro = []
    for i in range(num_puntos):
        angulo = 2 * np.pi * i / num_puntos
        x = centro_x + radio * np.cos(angulo)
        y = centro_y + radio * np.sin(angulo)
        z = centro_z
        puntos_aro.append([x, y, z])
    
    # Cerramos el círculo repitiendo el primer punto
    puntos_aro.append(puntos_aro[0])
    
    return np.array(puntos_aro, dtype='float32')

# Crear el aro
aro = crear_aro()

# dibujar una polilínea en 3D
def plot3(ax,c,color):
    x,y,z = c.T
    ax.plot(x,y,z,color)

def fun(a=-30, b = 30):
    fig = plt.figure(figsize=(8,7))
    ax = fig.add_subplot(111, projection='3d')
    plot3(ax,canasta_view,'r')
    plot3(ax, aro, 'orange')  # Aro en naranja
    plot3(ax,camline,'b')
    plot3(ax,esquina,'g')
    plot3(ax,esquina,'m')
    
    ax.set_zlim(0, ax.get_zlim()[1])
    
    ax.set_aspect('equal')
    ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
    ax.view_init(elev=b, azim=a)

0.5978739240404534


In [2]:
interactive(fun, a = (-180,180,10), b = (0,90,10))

interactive(children=(IntSlider(value=-30, description='a', max=180, min=-180, step=10), IntSlider(value=30, d…

In [3]:
from roboflow import Roboflow

rf = Roboflow(api_key="mf3cQuRUnk6Xaa6GWCNg")

In [4]:
project = rf.workspace("tfm-0225l").project("tfm-e54mm")


loading Roboflow workspace...
loading Roboflow project...


In [5]:
project.upload_dataset("C:/Users/TuUsuario/Documents/dataset")

AttributeError: 'Project' object has no attribute 'upload_dataset'